<br>

# Introdução

Para o presente projeto, é necessário fazer o *download* os dados espaciais contendo a divisão municipal. Optou-se por baixar os dados do [**Instituto Brasileiro de Geografia e Estatistica (IBGE)**](https://geoftp.ibge.gov.br/organizacao_do_territorio/malhas_territoriais/malhas_municipais), em escala 1:250.000, suficiente para atender o objetivo do projeto, que trata-se de demostrar quais as divisões administrativas, por município, de diversos órgãos e/ou instituições.

Portanto, faz-se o *download* do arquivo zipado do **IBGE** e se descompacta o arquivo _shapefile_. Aproveitou-se para renomear o arquivo um nome mais inteligível.

[Repo SP](https://github.com/michelmetran/sp)

In [ ]:
#!pip3 install openpyxl --upgrade

In [ ]:
import os
import requests
import pandas as pd
import geopandas as gpd

from osgeo import ogr
from zipfile import ZipFile

In [ ]:
from paths import *
from functions import *

<br>

# Dados Espaciais

<br>

## Limite Municipal 1:250.000

In [ ]:
# Input
url = 'https://geoftp.ibge.gov.br/organizacao_do_territorio/malhas_territoriais/malhas_municipais/municipio_2020/UFs/SP/SP_Municipios_2020.zip'
filename = 'sp_ibge_250k.zip'

# Get Data
get_download(url, filename, shps_path)
unzip(filename, shps_path)

In [ ]:
# Converte em *geojson*
file = 'sp_ibge_250k'
shp = os.path.join(shps_path, '{}.shp'.format(file))
gdf = gpd.read_file(shp)

# Deleta colunas
gdf.drop([
    'NM_MUN',
    'SIGLA_UF',
    'AREA_KM2'
],
    axis=1,
    inplace=True
)

# Renomeia colunas
gdf.rename(columns={'CD_MUN': 'id_municipio'}, inplace=True)

# Ajusta Tipo
gdf['id_municipio'] = gdf['id_municipio'].astype(int)

# Transforma Coordenadas
gdf = gdf.to_crs(epsg=4326)

# Results
gdf

In [ ]:
# Lê o arquivo csv com o nome dos municípios
df_mun = pd.read_csv(
    os.path.join(tabs_path, 'tab_municipio_nome.csv'),
    usecols=['id_municipio', 'municipio_nome']
)

# Merge
gdf_mun = pd.merge(
    df_mun,
    gdf,
    how='left',
    left_on='id_municipio',
    right_on='id_municipio'
)

# Results
gdf_mun

In [ ]:
# Converte para Geodataframe
gdf_mun = gpd.GeoDataFrame(
    gdf_mun,
    crs='EPSG:4326',
)

# Salva Arquivo
gdf_mun.to_file(    
    os.path.join(gjson_path, 'sp_250k_wgs84.geojson'),
    driver='GeoJSON',
    encoding='utf-8'
)

# Salva Arquivo
gdf_mun.to_file(
    os.path.join(gpkg_path, 'sp_250k_wgs84.gpkg'),
    layer='Limite Municipal',
    driver='GPKG',
    encoding='utf-8'
)

# Results
gdf_mun

<br>

## Limite Municipal 1:50.000

In [ ]:
# Input
url = 'http://datageo.ambiente.sp.gov.br/geoserver/datageo/LimiteMunicipal/wfs?version=1.0.0&request=GetFeature&outputFormat=SHAPE-ZIP&typeName=LimiteMunicipal'
filename = 'sp_igc_050k.zip'

# Get Data
get_download(url, filename, shps_path)
unzip(filename, shps_path)

In [ ]:
# Lê arquivo
file = 'sp_igc_050k'
shp = os.path.join(shps_path, '{}.shp'.format(file))
gdf = gpd.read_file(shp)

# Deleta colunas
gdf.drop(['Cod_Cetesb', 'Nome', 'Rotulo', 'UGRHI', 'Nome_ugrhi'], axis=1, inplace=True)

# Renomeia colunas
gdf.rename(
    columns={
        'Cod_ibge': 'id_municipio',
    },
    inplace=True
)

# Transforma Coordenadas
gdf = gdf.to_crs(epsg=4326)

# Results
gdf

In [ ]:
# Lê o arquivo csv com o nome dos municípios
df_mun = pd.read_csv(
    os.path.join(tabs_path, 'tab_municipio_nome.csv'),
    usecols=['id_municipio', 'municipio_nome']
)

# Merge
gdf = pd.merge(
    df_mun,
    gdf,
    how='left',
    left_on='id_municipio',
    right_on='id_municipio'
)

# Results
gdf

In [ ]:
# Converte para Geodataframe
gdf = gpd.GeoDataFrame(
    gdf,
    crs='EPSG:4326',
)

# Salva Arquivo
gdf.to_file(
    os.path.join(gjson_path, 'sp_050k_wgs84.geojson'),
    driver='GeoJSON',
    encoding='utf-8'
)

# Salva Arquivo
gdf.to_file(
    os.path.join(gpkg_path, 'sp_050k_wgs84.gpkg'),
    layer='Limite Municipal',
    driver='GPKG',
    encoding='utf-8'
)